In [ ]:
!pip install tensorflow-transform

INFO: pip is looking at multiple versions of tensorflow-transform to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-serving-api to determine which vers

In [ ]:
import tempfile
import pandas as pd
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam.impl as tft_beam

from tensorflow_transform.tf_metadata import schema_utils
from tensorflow_transform.tf_metadata import dataset_metadata

In [ ]:
dataset=pd.read_csv("pollution_small.csv")

In [ ]:
dataset.head()

,Date,pm10,no2,so2,soot
0,1/1/2009,98.67,14.10,44.38,34.81
1,1/2/2009,52.33,14.10,29.75,33.06
2,1/3/2009,74.67,20.50,36.25,39.25
3,1/4/2009,72.00,17.30,46.44,34.38
4,1/5/2009,81.00,25.64,56.56,45.59


In [ ]:
features=dataset.drop("Date",axis=1)

In [ ]:
features.head()

,pm10,no2,so2,soot
0,98.67,14.10,44.38,34.81
1,52.33,14.10,29.75,33.06
2,74.67,20.50,36.25,39.25
3,72.00,17.30,46.44,34.38
4,81.00,25.64,56.56,45.59


In [ ]:
dict_features=list(features.to_dict("index").values())

In [ ]:
dict_features[:2]

[{'pm10': 98.67, 'no2': 14.1, 'so2': 44.38, 'soot': 34.81},
 {'pm10': 52.33, 'no2': 14.1, 'so2': 29.75, 'soot': 33.06}]

In [ ]:
data_metadata=dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec({
        "pm10":tf.io.FixedLenFeature([],tf.float32),
        "no2":tf.io.FixedLenFeature([],tf.float32),
        "so2":tf.io.FixedLenFeature([],tf.float32),
        "soot":tf.io.FixedLenFeature([],tf.float32),
    })
)

In [ ]:
data_metadata

{'_schema': feature {
  name: "no2"
  presence {
    min_fraction: 1
  }
  shape {
  }
  type: FLOAT
}
feature {
  name: "pm10"
  presence {
    min_fraction: 1
  }
  shape {
  }
  type: FLOAT
}
feature {
  name: "so2"
  presence {
    min_fraction: 1
  }
  shape {
  }
  type: FLOAT
}
feature {
  name: "soot"
  presence {
    min_fraction: 1
  }
  shape {
  }
  type: FLOAT
}
, '_output_record_batches': True}

### Preprocessing Function

In [ ]:
def preprocessing_fn(inputs):
  pm10=inputs['no2']
  no2=inputs['no2']
  so2=inputs['so2']
  soot=inputs['soot']

  no2_normalized=no2 - tft.mean(no2)
  so2_normalized=so2 - tft.mean(so2)

  pm10_normalized=tft.scale_to_0_1(pm10)
  soot_normalized=tft.scale_by_min_max(soot)

  return {
      "no2_normalized":no2_normalized,
      "so2_normalized":so2_normalized,
      "pm10_normalized":pm10_normalized,
      "soot_normalized":soot_normalized
  }

In [ ]:
def data_transform():
  with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    transformed_dataset,transform_fn=((dict_features,data_metadata) | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn))

  transformed_data,transformed_metadata=transformed_dataset

  for i in range(len(transformed_data)):
    print("Raw:",dict_features[i])
    print("Transformed:",transformed_data[i])

In [ ]:
data_transform()

Raw: {'pm10': 98.67, 'no2': 14.1, 'so2': 44.38, 'soot': 34.81}
Transformed: {'no2_normalized': -18.57798194885254, 'pm10_normalized': 0.033990807831287384, 'so2_normalized': 28.85540771484375, 'soot_normalized': 0.283423513174057}
Raw: {'pm10': 52.33, 'no2': 14.1, 'so2': 29.75, 'soot': 33.06}
Transformed: {'no2_normalized': -18.57798194885254, 'pm10_normalized': 0.033990807831287384, 'so2_normalized': 14.225406646728516, 'soot_normalized': 0.26620757579803467}
Raw: {'pm10': 74.67, 'no2': 20.5, 'so2': 36.25, 'soot': 39.25}
Transformed: {'no2_normalized': -12.177982330322266, 'pm10_normalized': 0.0838855654001236, 'so2_normalized': 20.725406646728516, 'soot_normalized': 0.32710281014442444}
Raw: {'pm10': 72.0, 'no2': 17.3, 'so2': 46.44, 'soot': 34.38}
Transformed: {'no2_normalized': -15.377983093261719, 'pm10_normalized': 0.05893817916512489, 'so2_normalized': 30.9154052734375, 'soot_normalized': 0.27919331192970276}
Raw: {'pm10': 81.0, 'no2': 25.64, 'so2': 56.56, 'soot': 45.59}
Transfor